# Preprocessing des données

corpus de test : 

In [122]:
import pandas as pd
corpus_sephora3k = pd.read_csv("C:/Users/a.tekiouk/Sujet_2/Sujet_2/DATA/corpus_sephora3k.csv",sep=";", error_bad_lines=False)
corpus_sephora3k.head()
text = corpus_sephora3k[corpus_sephora3k['language'] == 'en']['text'].tolist()

C:\Users\a.tekiouk\AppData\Local\Temp\ipykernel_43584\735203815.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  corpus_sephora3k = pd.read_csv("C:/Users/a.tekiouk/Sujet_2/Sujet_2/DATA/corpus_sephora3k.csv",sep=";", error_bad_lines=False)







#### Identification des doublons
Métrique de distance permettant de mesurer la similarité entre deux posts

In [123]:
import textdistance
import numpy as np

In [125]:
hamming = textdistance.hamming.normalized_distance

On doit enlever les hashtags temporairements afin de vérifier s'il existe des doublons, car en effet des hashtag différent sur deux posts qui sont en fait identiques peuvent perturber le tri.

In [138]:
print(len(text))
import re
txt = text.copy()
for i in range(len(txt)-1):
    txt[i] = re.sub("#[A-Za-z0-9_]+","", txt[i])

print(len(txt))
        
        
hamming("lololo","kalalala")
    

1992
1992


0.75

In [139]:
#------------------------------------------------------------------------------------------
def dist_mat(t):
    dist_matrix = np.zeros((len(t),len(t)),dtype=np.float)
    for i in range(len(t)-1):
        for j in range(len(t)-1):
            dist_matrix[i,j] = hamming(t[i],t[j])
    return dist_matrix
#------------------------------------------------------------------------------------------

print(dist_mat(t))


[[0. 0.]
 [0. 0.]]


C:\Users\a.tekiouk\AppData\Local\Temp\ipykernel_43584\201382019.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dist_matrix = np.zeros((len(t),len(t)),dtype=np.float)


In [113]:
print(len(double))
for i in double:
    print(i)
    print("-----------------------------------")

460
💓 My first entry for the #benefitbrowsearch2023 ✨💓
benefitcosmetics 
✨ Using my FAV brow pencil the precisely my brow pencil shade- 4 to shade in my brows and give me a more natural look & to set my brows and make them fluffy the NEW fluff up brow wax ✨
••••••••••••
I need your help my babesssss for this years contest 💓
Just tag benefitcosmetics for me in the comments 🥹🫶🏼
I appreciate it so much! have lots of brow content coming your way ✨
•••••••••••••••
#benefitbrowsearch #benefitcosmetics #benefitbrows #cejas #makeup #maquillaje #makeup #browlamination #fluffybrows #browtutorial #makeupideas #explorepage #sephora #ulta
-----------------------------------
Nars radiant longwear Foundation instock

#nars #narscosmetics #elfhalo #elf #elfcosmetics #maybelline #sephora #sephoracollection #tarte #tartecosmetics #ootd #ootn #reels #uaeshoppinghaul #karachi #pakistan #pixi #glowrecipe
-----------------------------------
summerfridays sent over their Midnight Ritual Retinol a while back 

SyntaxError: invalid syntax (1548604841.py, line 1)

Suppression des posts identifiés comme doublons :

In [109]:
print(len(txt))

1984


#### Ratio mot/hashtag
Calcul du ratio mot/hastag dans un post

In [30]:
import spacy 
from spacy.lang.en import English

Ajout du token Hashtag dans la pipe (optionel)

In [65]:
from spacy.vocab import Vocab
from spacy.language import Language

@Language.component("hashtag")
def hashtag_pipe(doc):
    merged_hashtag = False
    while True:
        for token_index,token in enumerate(doc):
            if token.text == '#':
                if token.head is not None:
                    start_index = token.idx
                    end_index = start_index + len(token.head.text) + 1
                    print(start_index, end_index)
                    with doc.retokenize() as retokenizer:
                        if retokenizer.merge(doc[start_index:end_index]) is not None:
                            merged_hashtag = True
                            break
        if not merged_hashtag:
            break
        merged_hashtag = False
    return doc

nlp.add_pipe("hashtag")

<function __main__.hashtag_pipe(doc)>

Ratio Hashtag/mot

In [96]:
from spacy.lang.en import English
nlp = English()

i=0
junk = []
for i in range(len(text)-1):
    nb_hash = 0
    nb_word = 0
    doc = nlp(text[i])
    for token in doc:
        if(token.text == '#'):
            nb_hash+=1
        else:
            nb_word+=1
    if(nb_word-nb_hash==0):
        junk.append(text[i])
    elif(nb_hash/(nb_word-nb_hash) > 0.95):
        junk.append(text[i])
    

In [97]:
print(len(junk))
for t in junk:
    print(t)
    print("---------------------------------------------")

555
Absolutely stunning✨
.
.

#makeuplooks #makeupideas #beforeandafter #makeuptutorial #hudabeauty #makeupartistsworldwide #sephora #mac #juviasplace #anastasiabeverlyhills #nars #bhcosmetics #shopmissa #morphe #inglotcosmetics #bareminerals #results #blendtherules #dallasmakeupartist #makeupartists_worldwide #blendtherules #glamwithalaa
---------------------------------------------
Swatches de la palette Pro Pigment Volume 3 de norvinacosmetics 🧡
Une très bonne pigmentation 😍

#norvinacosmetics #anastasiabeverlyhills anastasiabeverlyhills #norvinapalette #norvina norvina #norvinavolume3 #norvinacollection #norvinavol3 #swatches #swatch #swatcheyeshadow #swatchpalette #eyeshadowpalettecollection #eyeshadowpalette #makeupcollection #makeupaddict #makeuplover #makeuplovers #makeupaddiction #instabeauty #instamakeup #sephora sephora #sephorafrance sephorafrance
---------------------------------------------
Eyes on point. 🤍✨
.
.
.
.
.
.
#makeup #makeupartist #makeuplover #makeupaddict #ma

TypeError: unsupported operand type(s) for -: 'list' and 'list'

Suppression des post ayant un ratio mot/hashtag trop faible :

1


#### Fréquence d'apparition dans la langue anglaise
Fréquence d'apparition de chaque mot du post dans la langue anglaise